In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from beam import GenerationMixin
import transformers
transformers.generation.utils.GenerationMixin = GenerationMixin
from transformers import AutoTokenizer, AutoModelForCausalLM,Gemma2ForCausalLM
import torch
from collections import defaultdict
from collections import Counter
import nltk
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# pip install accelerate


# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# outputs = model.generate(**input_ids, max_new_tokens=10,num_beams=2,num_return_sequences=2,return_dict_in_generate=True,output_scores=True)
# # for i in range(len(outputs)):
# #     print(outputs[i])
#     # print(tokenizer.decode(outputs[i],skip_special_tokens=True))
# # print(tokenizer.decode(outputs[0],skip_special_tokens=True))


In [4]:
sample_text='''
The final rites of renowned industrialist and philanthropist Ratan Tata were performed with full state honours at a central Mumbai crematorium on Thursday (October 10, 2024) evening.

The Mumbai police paid tribute to Tata with a gun salute.

The industry titan’s family members, including the half-brother Noel Tata, and top executives from the Tata Group like chairman N. Chandrasekaran, were present at the crematorium in Worli.

Industrialist Ratan Tata’s body was on Thursday morning taken from his house in a hearse, decked with white flowers, to the NCPA in south Mumbai where it was kept for people to pay their last respects.

Ratan Tata died at a Mumbai hospital late on Wednesday (October 9, 2024). The chairman emeritus of the Tata group was 86.

Tata was admitted to the Breach Candy Hospital in south Mumbai on Monday (October 7, 2024) owing to age-related issues, and to control his blood pressure.

Tributes poured in from across the country, following Tata’s death. Prime Minister Narendra Modi condoled his death, calling him a “visionary business leader, compassionate soul and an extraordinary human being”.

In a statement, the Congress said it “is deeply saddened by the passing of Padma Vibhushan Shri Ratan Tata, a titan of Indian industry and a philanthropist who shaped India’s corporate landscape.'''

In [5]:
# from sklearn.feature_extraction.text import CountVectorizer
# two_gram=CountVectorizer(ngram_range=(1,2),)
# x=two_gram.fit_transform(tokenizer.tokenize(sample_text),)
# print(two_gram.vocabulary_)
# x.toarray()

In [6]:
def kneser_ney_smoothing(ngram_counts, lower_order_counts, discount=0.75):
    """
    Apply Kneser-Ney smoothing to n-gram counts.

    Args:
        ngram_counts (Counter): Counts of n-grams (e.g., 4-grams or 3-grams).
        lower_order_counts (Counter): Counts of (n-1)-grams (e.g., 3-grams or 2-grams).
        discount (float): Discounting parameter.

    Returns:
        defaultdict: Smoothed probabilities.
    """
    continuation_counts = Counter()

    lower_counts = Counter()

    for ngram in ngram_counts:
        lower_ngram = ngram[1:]
        continuation_counts[lower_ngram] += 1
        lower_counts[lower_ngram] += 1

    def continuation_probability(word):
        return continuation_counts[word] / sum(continuation_counts.values())

    probabilities = defaultdict(lambda: defaultdict(float))

    for ngram, count in ngram_counts.items():
        lower_ngram = ngram[:-1]
        lower_count = lower_order_counts[lower_ngram]
        discounted_count = max(count - discount, 0)
        lambda_factor = (discount / lower_count) * len(continuation_counts)
        probabilities[lower_ngram][ngram[-1]] = (discounted_count / lower_count) + lambda_factor * continuation_probability(ngram[-1])

    return probabilities

def create_ngrams(tokens, n): return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]


def get_ngram_tokens(input_tokens, context_text,two_gram_probs, three_gram_probs,  four_gram_probs, top_k=4):

    print(input_tokens)
    context_tokens = nltk.word_tokenize(context_text)

    __token_pob__ = {}

    # print(probs,__token_pob__)
    num = 0
    while __token_pob__ == {} and num < 3:
        probs = [four_gram_probs, three_gram_probs, two_gram_probs][num]
        __inputs__ = tuple(input_tokens[-(3-num):])
        __token_pob__ = probs.get(__inputs__, {})
        num += 1

    sorted_tokens = sorted(__token_pob__.items(), key=lambda x: x[1], reverse=True)[:top_k]

    return [(tokenizer.convert_tokens_to_string([token]),prob,'n-gram') for token, prob in sorted_tokens]

In [7]:
context_tokens=tokenizer.tokenize(sample_text)

In [8]:


four_grams = create_ngrams(context_tokens, 4)
three_grams = create_ngrams(context_tokens, 3)
two_grams = create_ngrams(context_tokens, 2)
one_grams = create_ngrams(context_tokens, 1)

# Count the occurrences of each n-gram
four_gram_counts = Counter(four_grams)
three_gram_counts = Counter(three_grams)
two_gram_counts = Counter(two_grams)
one_gram_counts = Counter(one_grams)

# Apply Kneser-Ney smoothing iteratively
two_gram_probs = kneser_ney_smoothing(two_gram_counts, one_gram_counts)
three_gram_probs = kneser_ney_smoothing(three_gram_counts, two_gram_counts)
four_gram_probs = kneser_ney_smoothing(four_gram_counts, three_gram_counts)



In [9]:
# import numpy as np
# custom_vocab=list(two_gram_probs.keys())
# print(custom_vocab)
# custom_matrix=np.
# for i in custom_vocab:
#     custom_matrix.

In [10]:
two_gram_probs
class custom_ngram_model:
    def __init__(self, two_gram_probs, three_gram_probs, four_gram_probs):
        self.two_gram_probs = two_gram_probs
        self.three_gram_probs = three_gram_probs
        self.four_gram_probs = four_gram_probs

    def predict(self, input_tokens, context_text, top_k=4):
        return get_ngram_tokens(input_tokens, context_text, self.two_gram_probs, self.three_gram_probs, self.four_gram_probs, top_k=top_k)
#convert two_gram_probs to vocab and prob matrix

In [11]:
context_tokens

['\n',
 'The',
 '▁final',
 '▁rites',
 '▁of',
 '▁renowned',
 '▁industrial',
 'ist',
 '▁and',
 '▁philanthropist',
 '▁R',
 'atan',
 '▁Tata',
 '▁were',
 '▁performed',
 '▁with',
 '▁full',
 '▁state',
 '▁honours',
 '▁at',
 '▁a',
 '▁central',
 '▁Mumbai',
 '▁cre',
 'matorium',
 '▁on',
 '▁Thursday',
 '▁(',
 'October',
 '▁',
 '1',
 '0',
 ',',
 '▁',
 '2',
 '0',
 '2',
 '4',
 ')',
 '▁evening',
 '.',
 '\n\n',
 'The',
 '▁Mumbai',
 '▁police',
 '▁paid',
 '▁tribute',
 '▁to',
 '▁Tata',
 '▁with',
 '▁a',
 '▁gun',
 '▁salute',
 '.',
 '\n\n',
 'The',
 '▁industry',
 '▁titan',
 '’',
 's',
 '▁family',
 '▁members',
 ',',
 '▁including',
 '▁the',
 '▁half',
 '-',
 'brother',
 '▁Noel',
 '▁Tata',
 ',',
 '▁and',
 '▁top',
 '▁executives',
 '▁from',
 '▁the',
 '▁Tata',
 '▁Group',
 '▁like',
 '▁chairman',
 '▁N',
 '.',
 '▁Chand',
 'rase',
 'karan',
 ',',
 '▁were',
 '▁present',
 '▁at',
 '▁the',
 '▁cre',
 'matorium',
 '▁in',
 '▁Wor',
 'li',
 '.',
 '\n\n',
 'Industrial',
 'ist',
 '▁R',
 'atan',
 '▁Tata',
 '’',
 's',
 '▁body',
 '▁

In [12]:
input_text="Ratan Tata died"

In [13]:
x=get_ngram_tokens(tokenizer.tokenize(input_text), sample_text, two_gram_probs, three_gram_probs, four_gram_probs, top_k=4)
x

['R', 'atan', '▁Tata', '▁died']


[(' at', 0.25, 'n-gram')]

In [14]:
x=torch.tensor([1,2])
x.repeat(5,1)

tensor([[1, 2],
        [1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])

In [15]:
import numpy as np



n_steps = 30
n_beams = 100
length_penalty = 0
num_return_sequences = n_beams
n_grams={'two_gram_probs':two_gram_probs, 'three_gram_probs':three_gram_probs, 'four_gram_probs':four_gram_probs}


inputs = tokenizer([input_text], return_tensors="pt")
outputs = model.generate(
    **inputs,
    tokenizer=tokenizer,
    max_new_tokens=n_steps,
    num_beams=n_beams,
    num_return_sequences=min(num_return_sequences, n_beams),
    return_dict_in_generate=True,
    length_penalty=length_penalty,
    output_scores=True,
    do_sample=False,
    n_grams=n_grams
)


The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


beaming


/home/aseems/ml/mchl/beam.py:1945: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


depth:   0%|          | 0/35 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [16]:
decoded_sequences = tokenizer.batch_decode(outputs.sequences,skip_special_tokens=True)
beam_indices = outputs.beam_indices
step_beam_scores = outputs.step_beam_scores
n_indices = outputs.n_indices
step_beam_tokens = outputs.step_beam_tokens
input_tokens=input_text
step_parent_indices=outputs.step_parent_indices
step_extra_tokens_llm=outputs.step_extra_tokens
step_extra_scores_llm=outputs.step_extra_scores
step_extra_ngram_tokens=outputs.step_extra_ngram_tokens
step_extra_ngram_scores=outputs.step_extra_ngram_scores

In [17]:
decoded_sequences

['Ratan Tata died on April 16, 2023. He was chairman of Tata Sons from 1991 to 2012',
 'Ratan Tata died on April 16, 2023. He was chairman of the Tata Group from 1991 to 201',
 'Ratan Tata died ofOctober 18, 2022. He was Group Chairman of Tata Sons from 1991 to 201',
 'Ratan Tata died on April 16, 2023. He was chairman of Tata Sons from 1991 until 2012',
 'Ratan Tata died on April 16, 2023. He was chairman of the Tata Group from 1991 until 201',
 'Ratan Tata died on April 16, 2023. He was chairman of the Tata Group from 1991-2012',
 'Ratan Tata died ofOctober 18, 2022. He was Group Chairman of Tata Sons from 1991-2012',
 'Ratan Tata died on April 16, 2023. He was chairman of Tata Sons from 1991-2012.',
 'Ratan Tata died on April 16, 2023. He was chairman of Tata Sons from 1991 till 2012',
 'Ratan Tata died on April 16, 2023. He was chairman of Tata Sons from 1991 until his retirement in 2',
 'Ratan Tata died on February 23, 2023, at the age of 88.Ratan Tata was born on December 28',
 '

In [18]:
step_extra_tokens_llm

(([tensor(611),
   tensor(696),
   tensor(575),
   tensor(3646),
   tensor(576),
   tensor(13351),
   tensor(2001),
   tensor(235265),
   tensor(736),
   tensor(66509),
   tensor(235269),
   tensor(476),
   tensor(1452),
   tensor(235248),
   tensor(109),
   tensor(18901),
   tensor(578),
   tensor(7378),
   tensor(235292),
   tensor(3402),
   tensor(774),
   tensor(7910),
   tensor(10282),
   tensor(235341),
   tensor(573),
   tensor(675),
   tensor(10111),
   tensor(1378),
   tensor(1185),
   tensor(1317),
   tensor(8778),
   tensor(685),
   tensor(731),
   tensor(1861),
   tensor(14518),
   tensor(5245),
   tensor(4061),
   tensor(10500),
   tensor(2183),
   tensor(10317),
   tensor(7886),
   tensor(2290),
   tensor(577),
   tensor(108),
   tensor(2346),
   tensor(2149),
   tensor(591),
   tensor(235336),
   tensor(604),
   tensor(625),
   tensor(8084),
   tensor(901),
   tensor(1508),
   tensor(974),
   tensor(1794),
   tensor(18887),
   tensor(71024),
   tensor(671),
   tensor(115

In [19]:
step_extra_ngram_tokens

((['▁at'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  ['▁Monday', '▁Thursday', '▁Wednesday'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []),
 ([],
  ['▁a'],
  ['▁a'],
  ['▁a'],
  ['▁a'],
  ['▁a'],
  ['▁a'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['0', '4'],
  ['0', '4'],
  ['0', '4'],
  ['0', '4'],
  ['0', '4'],
  ['0', '4'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []),
 ([],
  ['▁Wor', '▁a', '▁from', '▁south'],
  ['▁Wor', '▁a', '▁from', '▁south'],
  ['▁Wor', '▁a', '▁from', '▁south'],
  ['▁Wor', '▁a', '▁from', '▁south'],
  ['▁Wor', '▁a', '▁from', '▁south'],
  ['▁Wor', '▁a', '▁from', '▁south'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [

In [20]:
import graphviz
from anytree import Node,AnyNode,RenderTree
import pickle
filename="tree"
dot = graphviz.Digraph(comment='Token Tree')
count=0
dot.node(str(count),input_tokens)
root=AnyNode(id=str(count),name=input_tokens,n_color="red",score=0,ngram_tokens=step_extra_ngram_tokens[0][0],
                        ngram_scores=step_extra_ngram_scores[0][0],llm_tokens=tokenizer.batch_decode(step_extra_tokens_llm[0][0]),
                        llm_scores=step_extra_scores_llm[0][0])

nodes=[]
nodes.append(root)
count+=1
level=0
for i in range(n_steps):
    if i==1:
        level+=1
    elif i>1:
        level+=len(step_beam_tokens)
    for j in range(len(step_beam_tokens)):
        beam=step_beam_tokens[j]
        next_token=tokenizer.decode(beam[i])
        if n_indices[j][i].item()==1:
            n_color="blue"
        else:
            n_color="green"
        # print(step_parent_indices[j][i].item()+level)
        # print(nodes[step_parent_indices[j][i].item()+level])
        if i<len(step_extra_ngram_tokens[0])-1:
            child=AnyNode(id=str(count),name=next_token,parent=nodes[step_parent_indices[j][i].item()+level],
                        n_color=n_color,score=step_beam_scores[j][i].item(),ngram_tokens=step_extra_ngram_tokens[j][i+1],
                        ngram_scores=step_extra_ngram_scores[j][i+1],llm_tokens=tokenizer.batch_decode(step_extra_tokens_llm[j][i+1]),
                        llm_scores=step_extra_scores_llm[j][i+1])
        else:
            child=AnyNode(id=str(count),name=next_token,parent=nodes[step_parent_indices[j][i].item()+level],
                        n_color=n_color,score=step_beam_scores[j][i].item(),ngram_tokens=[],
                        ngram_scores=[],llm_tokens=[],
                        llm_scores=[])
        
        nodes.append(child)
        dot.node(str(count),next_token,color=n_color)
        dot.edge(str(step_parent_indices[j][i].item()+level),str(count))
        count+=1
with open("tree.pkl","wb") as f:
    pickle.dump(root,f)
print(RenderTree(root))
dot.render(filename, format='png', cleanup=True)

AnyNode(id='0', llm_scores=[-1.2161617279052734, -1.5157337188720703, -2.56719970703125, -2.8156890869140625, -2.843698501586914, -3.6025447845458984, -3.610065460205078, -3.9184417724609375, -4.145753860473633, -4.272327423095703, -4.278354644775391, -4.355743408203125, -4.395393371582031, -4.77802848815918, -4.937288284301758, -5.068979263305664, -5.122392654418945, -5.140058517456055, -5.173189163208008, -5.181301116943359, -5.36750602722168, -5.422307968139648, -5.624797821044922, -5.751129150390625, -5.765203475952148, -5.8253631591796875, -5.909900665283203, -5.913185119628906, -5.929082870483398, -5.944591522216797, -6.008344650268555, -6.039899826049805, -6.058221817016602, -6.150876998901367, -6.173585891723633, -6.211996078491211, -6.221891403198242, -6.344728469848633, -6.399347305297852, -6.486183166503906, -6.496313095092773, -6.5379486083984375, -6.54243278503418, -6.576343536376953, -6.602178573608398, -6.669870376586914, -6.671758651733398, -6.69456672668457, -6.7025737

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.881331 to fit


'tree.png'

In [21]:
# import pickle

# # Save the tree object
# with open("tree.pkl", "wb") as f:
#     pickle.dump(dot, f)
#     print("Tree object pickled.")
